In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, pyautogui, re, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
driver = webdriver.Chrome()
driver.get("https://lactalis-smartit.onbmc.com/smartit/app/#/")
time.sleep(3)
pyautogui.hotkey("Esc")

In [3]:
def Verifica(tipo, valor:str):
    return WebDriverWait(driver, 10).until(EC.visibility_of_element_located((tipo, valor)))
    

In [4]:
Verifica(By.NAME, "AUTHENTICATION.LOGIN").send_keys("joaquim.junior.terceiro@br.lactalis.com")
driver.find_element(By.NAME, "AUTHENTICATION.PASSWORD").send_keys("L@ctali_S2024")
driver.find_element(By.NAME, "validateButton").click()

In [6]:
Verifica(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div/button').click()
time.sleep(2)
total_chamados = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[1]/div[3]/div[4]/h3/span').text
blocos_total = int(total_chamados)//75

In [9]:
def list_chamados(html):
    soup = BeautifulSoup(html, 'html.parser')
    chamados = soup.findAll('span', class_=re.compile('ng-binding ng-scope'))[7:]
    chamados_geral = []
    for chamado in chamados:
        chamados_geral.append(chamado.getText())
    return chamados_geral

def ResetBlocos(blocos_total):
    for bloco in range(0, blocos_total):
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()

In [12]:
chamados_tel = []
for bloco in range(0, blocos_total+1):
    if bloco == 0:
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text == 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').click()
        chamados_tel.append(list_chamados(driver.page_source))
    elif driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button').text != 'Próximo Bloco':
        driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[2]/button[2]').click()
        chamados_tel.append(list_chamados(driver.page_source))
ResetBlocos(blocos_total)

In [ ]:
def GerarDadosGeral():
    dados_chamados_geral = []
    
    for n in range(0, len(chamados_tel)):
        for chamado in chamados_tel[n][8::10]:
            dados_dict = {}
            for item in chamado.split('\n- ')[1:-5]:
                try:
                    chave, valor = item.split(':', 1)
                    dados_dict[chave.strip()] = valor.strip()
                except:
                    chave, valor = item.split('  ', 1)
                    dados_dict[chave.strip()] = valor.strip()
    
            dados_chamados_geral.append(dados_dict)
    return dados_chamados_geral

GerarDadosGeral()

In [13]:
dados_chamados_geral = []

for n in range(0, len(chamados_tel)):
    for chamado in chamados_tel[n][8::10]:
        dados_dict = {}
        for item in chamado.split('\n- ')[1:-5]:
            try:
                chave, valor = item.split(':', 1)
                dados_dict[chave.strip()] = valor.strip()
            except:
                chave, valor = item.split('  ', 1)
                dados_dict[chave.strip()] = valor.strip()
                
        
        # Converter o dicionário para JSON
        dados_json = json.dumps(dados_dict, indent=4, ensure_ascii=False)
        dados_chamados_geral.append(dados_dict)
        
pd.DataFrame(dados_chamados_geral)


,Nome do solicitante,Solicitado para,Site (Filial),Diretoria,Empresa,Solicitação de Computador,Motivo,Colaborador tem matrícula cadastrada?,Existe equipamento para uso no setor,Data Necessidade,...,Departamento,Data Planejada,Gestor da área,E-mail do gestor,Deseja substituir?,Marca do equipamento,Modelo do equipamento,Número do patrimônio,Boletim de ocorrência,Número do Chamado
0,GUSTAVO PEREIRA SILVA,GUSTAVO PEREIRA SILVA,BRA-084-UBERLANDIA,Industrial,Itambé,Notebook,Solicitar Novo Notebook,Sim,Não,2024-08-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RODOLFO VALENTIM DE MORAES,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,Tecnologia da Informação,Itambé,Notebook,Solicitar Novo Notebook,Não,Não,NaN,...,TECNOLOGIA DA INFORMAÇÃO,2024-08-05,Rodrigo de Alvarenga Borges,rodrigo.borges@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
2,Marilicia Michalski Ramos,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,Supply,Lactalis,Notebook,Solicitar Novo Notebook,Não,Não,NaN,...,NaN,2024-08-05,EDUARDO KROHT,eduardo.kroth@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
3,Marilicia Michalski Ramos,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,Supply,Lactalis,Notebook,Solicitar Novo Notebook,Não,Não,NaN,...,NaN,2024-08-05,EDUARDO KROHT,eduardo.kroth@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
4,RODOLFO VALENTIM DE MORAES,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,Tecnologia da Informação,Itambé,Notebook,Solicitar Novo Notebook,Não,Não,NaN,...,TECNOLOGIA DA INFORMAÇÃO,2024-08-05,Rodrigo de Alvarenga Borges,rodrigo.borges@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Alisson Muniz da Silva,Glaucio Souza,BRA-037-PORTO ALEGRE,Captação,Lactalis,Desktop,Substituição por recomendação técnica,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INC000000885548
278,Magali Konig,Magali Konig,BRA-052-TEUTÔNIA,Industrial,Lactalis,Desktop,Substituição por recomendação técnica,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643606
279,Magali Konig,Magali Konig,BRA-052-TEUTÔNIA,Industrial,Lactalis,Desktop,Nova Posição,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,Jessica Myrelli de Melo Queiroz Soare,Jessica Myrelli de Melo Queiroz Soare,BRA-009-BOM CONSELHO,Industrial,Lactalis,Notebook,Nova Posição,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[9::10])
    geral_chamado.extend(tel[8::10])
    status_chamado.extend(tel[7::10])
    tipo_chamado.extend(tel[6::10])
    designado_chamado.extend(tel[5::10])
    filial_chamado.extend(tel[4::10])
    cliente_chamado.extend(tel[3::10])
    criacao_chamado.extend(tel[2::10])
    n_chamado.extend(tel[1::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Designado': designado_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de Criacao': criacao_chamado,
})

df

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao
0,WO0000000347371,GUSTAVO PEREIRA SILVA,BRA-084-UBERLANDIA,LUANA DE ABREU RIBEIRO COUTINHO,847481 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:40:52
1,WO0000000347366,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,LUANA DE ABREU RIBEIRO COUTINHO,845378 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:17:05
2,WO0000000347280,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,NICOLAS VINICIUS MARTINS SILVA,846540 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 16:40:49
3,WO0000000347279,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,NICOLAS VINICIUS MARTINS SILVA,846543 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 16:40:47
4,WO0000000347165,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,LUANA DE ABREU RIBEIRO COUTINHO,845380 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:17:04
...,...,...,...,...,...,...,...
277,WO0000000247361,Alisson Muniz da Silva,BRA-032-MIRADOURO,NICOLAS VINICIUS MARTINS SILVA,652303 - BR_102 - Solicitação Computador,Designado,20 de set de 2023 10:37:38
278,WO0000000243868,Magali Konig,BRA-052-TEUTÔNIA,NICOLAS VINICIUS MARTINS SILVA,651593 - BR_102 - Solicitação Computador,Designado,8 de set de 2023 17:38:19
279,WO0000000243867,Magali Konig,BRA-052-TEUTÔNIA,NICOLAS VINICIUS MARTINS SILVA,651598 - BR_102 - Solicitação Computador,Designado,8 de set de 2023 17:38:16
280,WO0000000242427,Jessica Myrelli de Melo Queiroz Soare,BRA-009-BOM CONSELHO,NICOLAS VINICIUS MARTINS SILVA,648452 - BR_102 - Solicitação Computador,Designado,5 de set de 2023 11:08:47


In [ ]:
grupo_chamado = []
geral_chamado = []
status_chamado = []
tipo_chamado = []
designado_chamado = []
filial_chamado = []
cliente_chamado = []
criacao_chamado = []
n_chamado = []

for tel in chamados_tel:
    grupo_chamado.extend(tel[8::10])
    geral_chamado.extend(tel[7::10])
    status_chamado.extend(tel[6::10])
    tipo_chamado.extend(tel[5::10])
    designado_chamado.extend(tel[4::10])
    filial_chamado.extend(tel[3::10])
    cliente_chamado.extend(tel[2::10])
    criacao_chamado.extend(tel[1::10])
    n_chamado.extend(tel[::10])
    
df = pd.DataFrame({
    'Número do Chamado': n_chamado,
    'Cliente': cliente_chamado,
    'Filial': filial_chamado,
    'Grupo designado': grupo_chamado,
    'Tipo': tipo_chamado,
    'Status': status_chamado,
    'Data de criação': criacao_chamado,
})
 
df

In [ ]:
def dados_chamados(id:int):
    dados_chamado = []
    for tel in chamados_tel:
        dados_chamado.extend(tel[id::10])
    return dados_chamado

df = pd.DataFrame({
    'Número do Chamado': dados_chamados(0),
    'Cliente': dados_chamados(2),
    'Filial': dados_chamados(3),
    'Grupo designado': dados_chamados(8),
    'Tipo': dados_chamados(5),
    'Status': dados_chamados(6),
    'Data de criação': dados_chamados(1),
})

df

In [15]:
df_full = pd.concat([df, pd.DataFrame(dados_chamados_geral)], axis=1)
df_full

,Número do Chamado,Cliente,Filial,Designado,Tipo,Status,Data de Criacao,Nome do solicitante,Solicitado para,Site (Filial),...,Departamento,Data Planejada,Gestor da área,E-mail do gestor,Deseja substituir?,Marca do equipamento,Modelo do equipamento,Número do patrimônio,Boletim de ocorrência,Número do Chamado
0,WO0000000347371,GUSTAVO PEREIRA SILVA,BRA-084-UBERLANDIA,LUANA DE ABREU RIBEIRO COUTINHO,847481 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:40:52,GUSTAVO PEREIRA SILVA,GUSTAVO PEREIRA SILVA,BRA-084-UBERLANDIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WO0000000347366,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,LUANA DE ABREU RIBEIRO COUTINHO,845378 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:17:05,RODOLFO VALENTIM DE MORAES,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,...,TECNOLOGIA DA INFORMAÇÃO,2024-08-05,Rodrigo de Alvarenga Borges,rodrigo.borges@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
2,WO0000000347280,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,NICOLAS VINICIUS MARTINS SILVA,846540 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 16:40:49,Marilicia Michalski Ramos,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,...,NaN,2024-08-05,EDUARDO KROHT,eduardo.kroth@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
3,WO0000000347279,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,NICOLAS VINICIUS MARTINS SILVA,846543 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 16:40:47,Marilicia Michalski Ramos,Marilicia Michalski Ramos,BRA-037-PORTO ALEGRE,...,NaN,2024-08-05,EDUARDO KROHT,eduardo.kroth@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
4,WO0000000347165,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,LUANA DE ABREU RIBEIRO COUTINHO,845380 - BR_102 - Solicitação Computador,Designado,26 de jul de 2024 17:17:04,RODOLFO VALENTIM DE MORAES,RODOLFO VALENTIM DE MORAES,BRA-071-BELO HORIZONTE,...,TECNOLOGIA DA INFORMAÇÃO,2024-08-05,Rodrigo de Alvarenga Borges,rodrigo.borges@br.lactalis.com,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,WO0000000247361,Alisson Muniz da Silva,BRA-032-MIRADOURO,NICOLAS VINICIUS MARTINS SILVA,652303 - BR_102 - Solicitação Computador,Designado,20 de set de 2023 10:37:38,Alisson Muniz da Silva,Glaucio Souza,BRA-037-PORTO ALEGRE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INC000000885548
278,WO0000000243868,Magali Konig,BRA-052-TEUTÔNIA,NICOLAS VINICIUS MARTINS SILVA,651593 - BR_102 - Solicitação Computador,Designado,8 de set de 2023 17:38:19,Magali Konig,Magali Konig,BRA-052-TEUTÔNIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643606
279,WO0000000243867,Magali Konig,BRA-052-TEUTÔNIA,NICOLAS VINICIUS MARTINS SILVA,651598 - BR_102 - Solicitação Computador,Designado,8 de set de 2023 17:38:16,Magali Konig,Magali Konig,BRA-052-TEUTÔNIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,WO0000000242427,Jessica Myrelli de Melo Queiroz Soare,BRA-009-BOM CONSELHO,NICOLAS VINICIUS MARTINS SILVA,648452 - BR_102 - Solicitação Computador,Designado,5 de set de 2023 11:08:47,Jessica Myrelli de Melo Queiroz Soare,Jessica Myrelli de Melo Queiroz Soare,BRA-009-BOM CONSELHO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
dia = datetime.today().strftime('%d.%m')
df_full.to_excel(f'Chamados_br_1022_{dia}.xlsx', index=False)

In [ ]:
f'Chamados_br_71_{dia}.xlsx'